# Premier League 2021/22 Predictions

This model uses data from https://www.football-data.co.uk/englandm.php and https://www.kaggle.com/quadeer15sh/premier-league-standings-11-seasons-20102021, as well as https://en.wikipedia.org/wiki/2020%E2%80%9321_EFL_Championship

Present-time league data taken from https://footystats.org/england/premier-league on 2021/11/10

## Importing data
We will look at data from the past 5 years

In [155]:
import pandas as pd

csv_names = ["16_17", "17_18", "18_19", "19_20", "20_21", "21_22", "EPL Standings 2010-2021"]
CSVs = {}
path = "./Data/"

for name in csv_names:
     CSVs[name] = pd.read_csv(path + name + ".csv")

CSVs[csv_names[0]].head()

Div      Date        HomeTeam    AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  E0  13/08/16         Burnley     Swansea     0     1   A     0     0   D   
1  E0  13/08/16  Crystal Palace   West Brom     0     1   A     0     0   D   
2  E0  13/08/16         Everton   Tottenham     1     1   D     1     0   H   
3  E0  13/08/16            Hull   Leicester     2     1   H     1     0   H   
4  E0  13/08/16        Man City  Sunderland     2     1   H     1     0   H   

   ... BbAv<2.5  BbAH  BbAHh  BbMxAHH  BbAvAHH  BbMxAHA  BbAvAHA  PSCH  PSCD  \
0  ...     1.61    32  -0.25     2.13     2.06     1.86     1.81  2.79  3.16   
1  ...     1.52    33  -0.50     2.07     2.00     1.90     1.85  2.25  3.15   
2  ...     1.77    32   0.25     1.91     1.85     2.09     2.00  3.64  3.54   
3  ...     1.67    31   0.25     2.35     2.26     2.03     1.67  4.68  3.50   
4  ...     2.48    34  -1.50     1.81     1.73     2.20     2.14  1.25  6.50   

    PSCA  
0   2.89  
1   3.86  
2   2.16  
3   1.92  
4  14.50  

[5 rows x 65 columns]

## Linear regression

For linear regression, we will use standings from last season to predict the outcome of games

The features we want are:
- Games won last season
- Games drawn last season
- Games lost last season
- Promoted last season
- Goals for last season
- Goals against last season


First, we isolate the data from last season

In [156]:
csv_name = csv_names[-1]
csv = CSVs[csv_name]

data_last_season = csv.loc[csv['Season'] == "2020-21"]
data_last_season.head()

Season  Pos               Team  Pld   W   D   L  GF  GA  GD  Pts  \
200  2020-21    1    Manchester City   38  27   5   6  83  32  51   86   
201  2020-21    2  Manchester United   38  21  11   6  73  44  29   74   
202  2020-21    3          Liverpool   38  20   9   9  68  42  26   69   
203  2020-21    4            Chelsea   38  19  10   9  58  36  22   67   
204  2020-21    5     Leicester City   38  20   6  12  68  50  18   66   

                           Qualification or relegation  
200  Qualification for the Champions League group s...  
201  Qualification for the Champions League group s...  
202  Qualification for the Champions League group s...  
203  Qualification for the Champions League group s...  
204  Qualification for the Europa League group stag...

Next, we drop columns we don't want and reset the key

In [157]:
data_reduced = data_last_season.set_index("Team")

data_reduced = data_reduced.drop(columns = ["Season", "Pld", "GD", "Pts", "Qualification or relegation"], axis = 1)

data_reduced.head()

Pos   W   D   L  GF  GA
Team                                      
Manchester City      1  27   5   6  83  32
Manchester United    2  21  11   6  73  44
Liverpool            3  20   9   9  68  42
Chelsea              4  19  10   9  58  36
Leicester City       5  20   6  12  68  50

Now we have to add in the teams which were promoted, and drop the relegated teams. We will keep their stats from the Championship, but scale them by the number of games played. This will skew the results, so we will also add in a feature "Promoted" so the model can correct for this

In [159]:
data_promoted = data_reduced.assign(Promoted = False)

scale = 38 / 46

promoted = [
    {
        "Team": "Norwich City",
        "W": round(29 * scale),
        "D": round(10 * scale),
        "L": round(7 * scale),
        "GF": round(75 * scale),
        "GA": round(36 * scale),
        "Pos": 20,
        "Promoted": True
    },
    {
        "Team": "Watford",
        "W": round(27 * scale),
        "D": round(10 * scale),
        "L": round(9 * scale),
        "GF": round(63 * scale),
        "GA": round(30 * scale),
        "Pos": 20,
        "Promoted": True
    },
    {
        "Team": "Brentford",
        "W": round(24 * scale),
        "D": round(15 * scale),
        "L": round(7 * scale),
        "GF": round(79 * scale),
        "GA": round(42 * scale),
        "Pos": 20,
        "Promoted": True
    }
]

pmt_df = pd.DataFrame(promoted).set_index("Team")
pmt_data = data_promoted.append(pmt_df)
rel_df = pmt_data.drop(["Fulham", "West Bromwich Albion", "Sheffield United"])
data_complete = rel_df
data_complete

Pos   W   D   L  GF  GA  Promoted
Team                                                      
Manchester City            1  27   5   6  83  32     False
Manchester United          2  21  11   6  73  44     False
Liverpool                  3  20   9   9  68  42     False
Chelsea                    4  19  10   9  58  36     False
Leicester City             5  20   6  12  68  50     False
West Ham United            6  19   8  11  62  47     False
Tottenham Hotspur          7  18   8  12  68  45     False
Arsenal                    8  18   7  13  55  39     False
Leeds United               9  18   5  15  62  54     False
Everton                   10  17   8  13  47  48     False
Aston Villa               11  16   7  15  55  46     False
Newcastle United          12  12   9  17  46  62     False
Wolverhampton Wanderers   13  12   9  17  36  52     False
Crystal Palace            14  12   8  18  41  66     False
Southampton               15  12   7  19  47  68     False
Brighton & Hove Albion    16   9  14  15  40  46     False
Burnley                   17  10   9  19  33  55     False
Norwich City              20  24   8   6  62  30      True
Watford                   20  22   8   7  52  25      True
Brentford                 20  20  12   6  65  35      True

Next we must normalize our numerical data, and extract our y value

In [185]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numerical_cols = ["W", "D", "L", "GF", "GA"]
data_norm = data_complete
data_norm[numerical_cols] = data_complete[numerical_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

y_column = "Pos"
y = data_norm[y_column].to_frame()
X = data_norm.loc[:, data_norm.columns != y_column]
X

W         D         L    GF        GA  \
Team                                                                    
Manchester City          1.000000  0.000000  0.000000  1.00  0.162791   
Manchester United        0.666667  0.666667  0.000000  0.80  0.441860   
Liverpool                0.611111  0.444444  0.230769  0.70  0.395349   
Chelsea                  0.555556  0.555556  0.230769  0.50  0.255814   
Leicester City           0.611111  0.111111  0.461538  0.70  0.581395   
West Ham United          0.555556  0.333333  0.384615  0.58  0.511628   
Tottenham Hotspur        0.500000  0.333333  0.461538  0.70  0.465116   
Arsenal                  0.500000  0.222222  0.538462  0.44  0.325581   
Leeds United             0.500000  0.000000  0.692308  0.58  0.674419   
Everton                  0.444444  0.333333  0.538462  0.28  0.534884   
Aston Villa              0.388889  0.222222  0.692308  0.44  0.488372   
Newcastle United         0.166667  0.444444  0.846154  0.26  0.860465   
Wolverhampton Wanderers  0.166667  0.444444  0.846154  0.06  0.627907   
Crystal Palace           0.166667  0.333333  0.923077  0.16  0.953488   
Southampton              0.166667  0.222222  1.000000  0.28  1.000000   
Brighton & Hove Albion   0.000000  1.000000  0.692308  0.14  0.488372   
Burnley                  0.055556  0.444444  1.000000  0.00  0.697674   
Norwich City             0.833333  0.333333  0.000000  0.58  0.116279   
Watford                  0.722222  0.333333  0.076923  0.38  0.000000   
Brentford                0.611111  0.777778  0.000000  0.64  0.232558   

                         Promoted  
Team                               
Manchester City             False  
Manchester United           False  
Liverpool                   False  
Chelsea                     False  
Leicester City              False  
West Ham United             False  
Tottenham Hotspur           False  
Arsenal                     False  
Leeds United                False  
Everton                     False  
Aston Villa                 False  
Newcastle United            False  
Wolverhampton Wanderers     False  
Crystal Palace              False  
Southampton                 False  
Brighton & Hove Albion      False  
Burnley                     False  
Norwich City                 True  
Watford                      True  
Brentford                    True

Finally, we are ready to perform our regression

In [186]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

print(regressor.intercept_)

print(regressor.coef_)

[8.63821787]
[[-8.1727392   0.08173843 11.25951229 -0.79039077 -4.1396513  19.08493344]]


Let's see if that worked

In [187]:
y_pred = regressor.predict(X_test)
results = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred.flatten()}, index=y_test.index)
results

Actual  Predicted
Team                                
Watford                20  22.413631
Manchester United       2   0.782756
Brentford              20  21.323714
Leeds United            9   9.096611

Not bad! Now let's see what our model predicts based on this year's form

In [203]:
test_data_this_season = csv.loc[csv['Season'] == "2021-22"]
test_reduced = test_data_this_season.set_index("Team").drop(columns = ["Season", "Pld", "GD", "Pts", "Qualification or relegation"], axis = 1)
test_promoted = pd.concat([test_reduced, data_norm["Promoted"].to_frame()], axis = 1)

test_norm = test_promoted
test_norm[numerical_cols] = test_promoted[numerical_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

X_test_data = test_norm.loc[:, test_norm.columns != y_column]
X_test_data
y_pred_21_22 = regressor.predict(X_test_data)
results = pd.DataFrame({'Predicted': y_pred_21_22.flatten()}, index=test_norm.index)
results = results.sort_values('Predicted')
results

Predicted
Team                              
Chelsea                  -0.186966
Liverpool                 0.450154
West Ham United           0.871233
Manchester City           2.218795
Arsenal                   3.805324
Brighton & Hove Albion    4.507616
Crystal Palace            5.078025
Manchester United         5.500356
Leicester City            6.441329
Everton                   6.817661
Southampton               7.198509
Tottenham Hotspur         7.584673
Wolverhampton Wanderers   8.276537
Leeds United              8.669208
Burnley                  11.487468
Aston Villa              11.940199
Newcastle United         12.770001
Brentford                29.000220
Watford                  31.274098
Norwich City             33.837768

We didn't bound the output to a 1 to 20 scale so it's a bit off, but we can infer the answer from the order.

It looks like bad news for my team, Tottenham Hotspur! Maybe we'll have to try a different model...